In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Utils
from src.utils.analysis_basics.check_gen_deadlines import plotfig_empirical_scheduling_window_distributions

# Simulation
from src.simulation.ProductionSimulation import ProductionSimulation

pd.set_option('display.min_rows', 21)
pd.set_option('display.max_rows', 25)


ImportError: cannot import name 'plotfig_empirical_flow_budget_distributions' from 'src.utils.analysis_basics.check_gen_deadlines' (C:\Users\konra\Coding\HTWD_Minimalinvasives_Job-Shop_Scheduling\src\utils\analysis_basics\check_gen_deadlines.py)

In [ ]:
from decimal import Decimal
from src.domain.Collection import LiveJobCollection
from src.domain.Query import JobQuery
# from src.domain.Initializer import MachineInitializer, DataSourceInitializer
from src.DataFrameEnrichment import DataFrameEnrichment

In [ ]:
source_name= "Fisher and Thompson 10x10"
max_bottleneck_utilization=Decimal("0.85")

# I. Deadlines

In [ ]:
jobs = JobQuery.get_by_source_name_and_max_bottleneck_utilization(
    source_name=source_name,
    max_bottleneck_utilization=max_bottleneck_utilization
)

# Collection with jobs
jobs_collection = LiveJobCollection(jobs)
jobs_collection.to_jobs_dataframe()

## A) Simulation

In [ ]:
simulation = ProductionSimulation(sigma = 0, verbose = False, with_earliest_start=True)
simulation.run(jobs_collection, start_time = 0, end_time = None)

In [ ]:
finished_operations = simulation.get_finished_operation_collection()
df_fcfs = finished_operations.to_operations_dataframe()
df_fcfs


In [ ]:
#if day_count <= 5:
#    gantt.get_plot(df_fcfs_execution, perspective="Machine")

In [ ]:
df_jobs_times_temp = finished_operations.to_jobs_metrics_dataframe()
df_jobs_times_temp


In [ ]:
plotfig_empirical_flow_budget_distributions(df_jobs_times_temp, due_date_column="End", y_max=0.0015)
plt.show()

## B) Generation of deadlines using log-normal distribution

In [ ]:
df_jobs_times = DataFrameEnrichment.add_groupwise_lognormal_deadlines_by_group_mean(df_jobs_times_temp, sigma=0.25)
df_jobs_times

####  Plot of the new deadlines

In [ ]:
plotfig_empirical_flow_budget_distributions(df_jobs_times, y_max=0.0015)
plt.show()

#### Ensuring that the processing time is achievable to a certain extent in the best-case scenario

In [ ]:
# min. 100 % of processing time for each job is covered in best-case
df_jobs_times_final = DataFrameEnrichment.ensure_reasonable_deadlines(df_jobs_times, min_coverage = 1.0)

diff_count = (df_jobs_times_final["Deadline"] > df_jobs_times["Deadline"]).sum()

# Info: Compare the "Deadline" column
print(f"Number of different entries in 'Deadline': {diff_count}")

df_jobs_times_final

In [ ]:
# --- Update in DB ---
"""
JobQuery.update_job_deadlines_from_df(
    df = df_jobs_times_final,
    job_column="Job",
    deadline_column = "Deadline"
)
"""
None

# II. Transition Times

In [ ]:
df_fcfs

In [ ]:
# noinspection PyProtectedMember
df_jobs_transition_times = DataFrameEnrichment._compute_transition_times_per_job_backward(df_fcfs)
df_jobs_transition_times     

In [ ]:
df_avg_transition_times = DataFrameEnrichment.compute_avg_transition_times_per_machine_backward(df_fcfs)
df_avg_transition_times

In [ ]:
# --- Add to DB ---

""""
MachineInitializer.insert_from_dataframe(
    df = df_avg_transition_times,
    source_name= source_name,
    max_bottleneck_utilization=max_bottleneck_utilization,
)
"""
None

### ____________________________________________________________________________________